<a href="https://colab.research.google.com/github/Arhammanj/Arhammanj/blob/main/Qlearning_for_tesla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving archive.zip to archive.zip


In [ ]:
import zipfile
import os
import pandas as pd

# Extract the ZIP file
with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/tesla_data")

# List files to see CSV name
files_in_folder = os.listdir("/content/tesla_data")
print("Extracted files:", files_in_folder)

# Load the CSV (if there’s only one CSV in the zip)
csv_file = "/content/tesla_data/" + files_in_folder[0]
df = pd.read_csv(csv_file)

# Prepare the DataFrame
df['Date'] = pd.to_datetime(df['Date'])  # convert date column
df = df.sort_values('Date')              # sort by date
df.reset_index(drop=True, inplace=True)




Extracted files: ['TSLA.csv']


In [ ]:
import numpy as np
import pandas as pd
import random


In [ ]:
def get_state(price_today, price_yesterday, holding):
    trend = 1 if price_today > price_yesterday else 0
    return trend * 2 + holding  # encodes to 0-3


In [ ]:
actions = ['BUY', 'SELL', 'HOLD']
num_actions = len(actions)


In [ ]:
num_states = 4  # 2 trend * 2 holding
Q = np.zeros((num_states, num_actions))


In [ ]:
alpha = 0.1     # learning rate
gamma = 0.95    # discount factor (future reward importance)
epsilon = 0.2   # exploration probability
episodes = 50   # number of training runs


In [ ]:
prices = df['Close'].values

for episode in range(episodes):
    holding = 0      # 0 = not holding stock, 1 = holding
    buy_price = 0    # price at which we bought
    total_profit = 0

    for t in range(1, len(prices)):

        state = get_state(prices[t], prices[t-1], holding)

        # ε-greedy: decide explore or exploit
        if random.random() < epsilon:
            action = random.randint(0, num_actions - 1)  # random action
        else:
            action = np.argmax(Q[state])                # best known action

        reward = 0

        # Execute action
        if action == 0 and holding == 0:      # BUY
            holding = 1
            buy_price = prices[t]

        elif action == 1 and holding == 1:    # SELL
            reward = prices[t] - buy_price   # profit/loss
            total_profit += reward
            holding = 0

        elif action == 2:                      # HOLD
            reward = 0

        # Next state
        next_state = get_state(prices[t], prices[t-1], holding)

        # Update Q-table
        Q[state, action] = Q[state, action] + alpha * (
            reward + gamma * np.max(Q[next_state]) - Q[state, action]
        )

    print(f"Episode {episode+1}/{episodes} - Total Profit: {total_profit:.2f}")


Episode 1/50 - Total Profit: -10.42
Episode 2/50 - Total Profit: 524.42
Episode 3/50 - Total Profit: 361.08
Episode 4/50 - Total Profit: 468.48
Episode 5/50 - Total Profit: 53.10
Episode 6/50 - Total Profit: 981.67
Episode 7/50 - Total Profit: 314.74
Episode 8/50 - Total Profit: 92.30
Episode 9/50 - Total Profit: 756.26
Episode 10/50 - Total Profit: -127.56
Episode 11/50 - Total Profit: 417.95
Episode 12/50 - Total Profit: 606.98
Episode 13/50 - Total Profit: 711.49
Episode 14/50 - Total Profit: 1123.29
Episode 15/50 - Total Profit: 784.46
Episode 16/50 - Total Profit: 563.71
Episode 17/50 - Total Profit: 937.89
Episode 18/50 - Total Profit: 39.38
Episode 19/50 - Total Profit: 740.42
Episode 20/50 - Total Profit: 916.70
Episode 21/50 - Total Profit: 451.44
Episode 22/50 - Total Profit: 346.72
Episode 23/50 - Total Profit: 494.67
Episode 24/50 - Total Profit: -64.76
Episode 25/50 - Total Profit: 655.12
Episode 26/50 - Total Profit: 804.61
Episode 27/50 - Total Profit: 446.71
Episode 28/

In [ ]:
print("Q-table after training:")
for s in range(num_states):
    print(f"State {s}: {Q[s]}")


Q-table after training:
State 0: [4.55921165e-07 5.55345184e-06 5.56076377e-06]
State 1: [ 2.88970070e-07 -4.29587851e+01  2.89510961e-07]
State 2: [29.34885434 14.04735056 16.51632588]
State 3: [22.12151054 42.03162529 18.70030873]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
holding = 0
buy_price = 0
profit = 0

for t in range(1, len(prices)):
    state = get_state(prices[t], prices[t-1], holding)
    action = np.argmax(Q[state])

    if action == 0 and holding == 0:      # BUY
        holding = 1
        buy_price = prices[t]
        print(f"Buy at {prices[t]:.2f} on day {t}")

    elif action == 1 and holding == 1:    # SELL
        profit += prices[t] - buy_price
        holding = 0
        print(f"Sell at {prices[t]:.2f} on day {t}")

print(f"Total simulated profit: {profit:.2f}")


Buy at 3.49 on day 6
Sell at 3.63 on day 9
Buy at 3.97 on day 10
Sell at 3.98 on day 11
Buy at 4.13 on day 12
Sell at 4.38 on day 13
Buy at 4.20 on day 16
Sell at 4.26 on day 17
Buy at 4.14 on day 20
Sell at 4.18 on day 23
Buy at 4.39 on day 24
Sell at 3.92 on day 28
Buy at 3.66 on day 32
Sell at 3.76 on day 33
Buy at 3.83 on day 34
Sell at 3.76 on day 36
Buy at 3.82 on day 37
Sell at 4.03 on day 38
Buy at 3.98 on day 40
Sell at 3.97 on day 43
Buy at 4.09 on day 45
Sell at 4.21 on day 46
Buy at 4.18 on day 49
Sell at 4.14 on day 52
Buy at 4.22 on day 53
Sell at 4.40 on day 54
Buy at 4.21 on day 57
Sell at 4.02 on day 61
Buy at 4.11 on day 62
Sell at 4.28 on day 63
Buy at 4.40 on day 64
Sell at 4.12 on day 66
Buy at 4.20 on day 67
Sell at 4.22 on day 68
Buy at 4.11 on day 74
Sell at 4.15 on day 75
Buy at 4.13 on day 79
Sell at 4.15 on day 80
Buy at 4.17 on day 82
Sell at 4.27 on day 83
Buy at 4.24 on day 85
Sell at 4.37 on day 86
Buy at 4.35 on day 89
Sell at 4.98 on day 90
Buy at 5.00 

core steps of Q-Learning applied to any dataset

Get the data (environment)

Download CSV or get data from your source.

Example: Tesla stock prices, maze grid, robot sensors.

Prepare the DataFrame (or environment representation)

Clean the data (sort by date, remove missing values, etc.).

Convert it into a form the agent can use (states and features).

Define states, actions, and rewards

State: snapshot of environment (e.g., trend + holding for stock).

Action: what the agent can do (e.g., buy/sell/hold).

Reward: feedback from environment (profit, reaching goal, etc.).

Create Q-table

Rows = all possible states

Columns = all possible actions

Cells = expected reward for taking that action in that state

Initially, all cells = 0

Set hyperparameters

Learning rate (α), discount factor (γ), epsilon (ε), number of episodes

Train the agent (loop over episodes)

For each day/step:

Observe current state

Choose action:

With probability ε → explore (random action)

Otherwise → exploit (best known action from Q-table)

Execute the action → get reward

Observe next state

Update Q-table using Q-learning formula

Repeat for many episodes until Q-table converges

The agent learns which action gives the highest expected reward for each state